In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, os
import pandas as pd 

path = os.getcwd() + '/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(chrome_options=options, executable_path=path)
    
def cria_csv(dicionario):
    df = pd.DataFrame.from_dict(dicionario)
    df.to_csv('animes.csv')

In [4]:
from selenium.common.exceptions import NoSuchElementException

def lista_completa_de_animes(driver=driver):
    nome = []
    genero = []
    dtAtualizacao = []
    nEpisodiosTotal = []
    nEpisodios = []
    status = []
    img = []
    link = []
    
    driver.get('https://punchsubs.net/principal') #pagina inicial
    driver.find_element_by_xpath('//*[@id="main-menu"]/ul/li[1]/a').click()
    pgns = len(driver.find_elements_by_xpath('/html/body/div[4]/div[1]/div[4]/div[4]/div/div[2]/ul/li'))
    for i in range(pgns):     
        while True:
            try:
                driver.refresh()
                urls = driver.find_elements_by_xpath('/html/body/div[4]/div[1]/div[4]/div[4]/div/div[2]/ul/li')
                url = urls[i].find_element_by_xpath('./a')
                url.click()
                break
            except:
                continue

        elementos = driver.find_elements_by_xpath('//*[@id="projetosListagem"]/ul/li')
        for elemento in elementos:
            anime = {'nome': [],
                     'genero': [],
                     'dtAtualizacao': [],
                     'nEpisodiosTotal': [],
                     'nEpisodios': [],
                     'status': [],
                     'img': [],
                     'link': []}

            try:
                nome.append(elemento.find_element_by_xpath('./div[2]/div[1]/b/a').text)
                g = elemento.find_element_by_xpath('./div[2]/span[1]').text
                genero.append(g.replace('Gênero: ', ''))
                dtA = elemento.find_element_by_xpath('./div[2]/span[2]').text
                dtAtualizacao.append(dtA.replace('Última Atualização: ', ''))
                eps = elemento.find_element_by_xpath('./div[2]/span[3]').text
                eps = eps.replace('Número de Episódios: ', '')
                eps = eps.split('/')
                eps = [int(ep) for ep in eps]
                nEpisodiosTotal.append(eps[0])
                nEpisodios.append(eps[1])
                st =  elemento.find_element_by_xpath('./div[2]/span[4]').text
                status.append(st.replace('Status: ', ''))
                img.append(elemento.find_element_by_xpath('./div[1]/a/img[@src]').get_attribute('src'))
                lk = elemento.find_element_by_xpath('./div[1]/a[@href]').get_attribute('href')
                lk = lk.replace('fullhd', 'stream')
                lk = lk.replace('hd', 'stream')
                lk = lk.replace('sd', 'stream')
                lk = lk.replace('mq', 'stream')
                link.append(lk)
                
                print('Anime: {}'.format(elemento.find_element_by_xpath('./div[2]/div[1]/b/a').text))

            except NoSuchElementException:
                pass
    
    anime['nome'] = nome
    anime['genero'] = genero
    anime['dtAtualizacao'] = dtAtualizacao
    anime['nEpisodiosTotal'] = nEpisodiosTotal
    anime['nEpisodios'] = nEpisodios
    anime['status'] = status
    anime['img'] = img
    anime['link'] = link
    return anime

In [5]:
def cria_csv(dicionario, csv):
    df = pd.DataFrame(dicionario)
    df.to_csv(csv)

In [6]:
def captura_espisodios(driver, url):
    driver.get(url)
    nome = []
    episodio = []
    link = []
    episodios = {'nome': [],
                 'episodio': [],
                 'link': []}

    pgns = len(driver.find_elements_by_xpath('//*[@id="load"]/div[2]/ul/li'))
    for i in range(pgns):     
        while True:
            try:
                driver.refresh()
                urls = driver.find_elements_by_xpath('//*[@id="load"]/div[2]/ul/li')
                url = urls[i].find_element_by_xpath('./a')
                url.click()
                break
            except:
                continue

        trs = driver.find_elements_by_xpath('//*[@id="listagemDownloads"]/table/tbody/tr')
        for tr in trs:
            tds = tr.find_elements_by_xpath('./td')
            for td in tds:
                try:
                    episodio.append(td.find_element_by_xpath('./span').text)
                    link.append(td.find_element_by_xpath('./div[2]/a[@href]').get_attribute('href'))
                    nome.append(driver.find_element_by_xpath('//*[@id="load"]/div[1]/div[1]/div[2]/b').text)
                    
                    print('{} - {}'.format(driver.find_element_by_xpath('//*[@id="load"]/div[1]/div[1]/div[2]/b').text, 
                                          td.find_element_by_xpath('./span').text))

                except NoSuchElementException:
                    pass

    episodios['nome'] = nome
    episodios['episodio'] = episodio
    episodios['link'] = link
    return episodios

#atualiza os links com o link real do episodio
def atualiza_link(driver, episodios):
    links = episodios['link']
    new_links = []
    for link in links:
        driver.get(link)
        wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="player_frame"]')))
        iframe = driver.find_element_by_xpath('//*[@id="player_frame"]')
        driver.switch_to_frame(iframe)
        new_links.append(driver.find_element_by_xpath('/html/body/div[2]/div/video[@src]').get_attribute('src'))

    episodios['link'] = new_links
    return episodios

##### Cria o arquivo animes.csv *com todos os animes da página

In [ ]:
cria_csv(lista_completa_de_animes(), 'animes.csv')

##### crie um csv a parte com os animes que deseja sempre atualizar os episodios
##### importe o csv com a função abaixo, ela ira instanciar na variável um dicionario

In [ ]:
df = dict(pd.read_csv('animes.csv', sep=';', usecols=['nome', 'img', 'link']))

for i in range(len(df['nome'])):
    csv = df['nome'][i] + '.csv'
    episodios = atualiza_link(driver, captura_espisodios(driver, df['link'][i]))

In [20]:
cria_csv(episodios, csv)

In [34]:
def cria_lista_m3u(dicionario):
    lista = ['#EXTM3U\n\n']
    for i in range(len(dicionario['nome'])):
        lista.append('#EXTINF:-1 tvg-id="" tvg-logo="" group-title="{}", {}\n{}\n\n'.format(
        dicionario['nome'][i], dicionario['episodio'][i], dicionario['link'][i]))   
    
    nm = dicionario['nome'][0] + '.m3u' 
    doc = open(nm, 'w')
    
    for item in lista:
        doc.write(item)
    
    doc.close()

In [35]:
df = dict(pd.read_csv('ZETMAN.csv'))

In [36]:
cria_lista_m3u(df)